<a href="https://colab.research.google.com/github/amiruladlil19/Chatbot-projects/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import json
import random
import pickle
import tensorflow
import tflearn
import numpy
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory #install sastrawi first
from tensorflow.python.framework import ops
ops.reset_default_graph()
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [52]:
with open('intents2.json') as file:
    data = json.load(file)

In [53]:
nltk.download('punkt')

words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])

words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)


training = numpy.array(training)
output = numpy.array(output)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [54]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

In [55]:
try:
    model.load("model.tflearn")
except:
    model.fit(training, output, n_epoch=500, batch_size=8, show_metric=True)
    model.save("model.tflearn")

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [56]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    return numpy.array(bag)


order = []


def listToString(order):

    str1 = ""

    for ele in order:
        str1 += " "+ele

    return str1


def chat():
    print("Ada yang bisa saya bantu?")
    while True:
        inp = input("Anda: ")

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        if tag == "keluar":
            if len(order) == 1:
                print("Anda bisa menemukan layanan tersebut di menu utama kami :)")
                print("Sampai jumpa! Semoga hari Anda menyenangkan!")
                break
            elif len(order) == 0:
                print("Semoga hari Anda menyenangkan!")
                break
            else:
                reccomend(listToString(order))
                print("Sampai jumpa! Semoga hari Anda menyenangkan!")
                break

        if tag == 'cat':
            order.append('memulas')

        if tag == 'atap':
            order.append('bumbung')

        if tag == 'lantai':
            order.append('ubin')
          
       

        for tg in data["intents"]:

            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))
    


def reccomend(inp):
    results = model.predict([bag_of_words(inp, words)])
    results_index = numpy.argmax(results)
    tag = labels[results_index]
    for tg in data["intents"]:
        if tg['tag'] == tag:
            responses = tg['responses']

    print(random.choice(responses))

In [57]:
chat()


Ada yang bisa saya bantu?
Anda: kamu siapa>
Saya adalah chatbot yang akan merekomendasikan layanan yang Anda butuhkan. Ada yang bisa saya bantu?
Anda: tidak adakmle
Semoga hari Anda menyenangkan!


In [58]:
len(order)

0